In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Kkma
kkma = Kkma()
from collections import Counter
from google.colab import drive
import os

In [3]:
# 구글 드라이브 마운트
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# 파일 접두어 정의
file_prefixes = ["국가애도기간", "대통령사과", "경찰책임", "정부책임", "유가족협의회", "특별법"]

# 디렉토리 경로
directory_path = "/content/gdrive/MyDrive/6_emotion/"

dfs=[]

# 각 파일 접두어에 대해 반복
for prefix in file_prefixes:
    # 파일 경로 정의
    file_path = directory_path + f"{prefix}_emotion.xlsx"

    df = pd.read_excel(file_path)

    dfs.append(df)

In [5]:
def extract_and_merge_nouns(text):
    # 명사 추출
    nouns = okt.nouns(text)

    if len(nouns) == 0:
      return []

    # 연속된 명사들을 하나로 합치기
    merged_nouns = [nouns[0]]
    i = 1
    j = 0
    while i < len(nouns):
        now_noun = merged_nouns[j]
        next_noun = nouns[i]

        now_index = text.find(now_noun)
        now_length = len(now_noun)
        next_index = text.find(next_noun)

        if next_index == now_index + now_length:
          merged_nouns[j] = now_noun + next_noun
        else:
          merged_nouns.append(next_noun)
          j += 1

        i += 1

    #크기가 1인 명사 제거
    merged_nouns = [noun for noun in merged_nouns if len(noun) > 1]
    #merged_nouns = [noun for noun in nouns if len(noun) > 1]
    return merged_nouns


In [9]:
desired_pos_tags = ['NNG', 'NNP']

def extract_and_merge_nouns(text):
    try:
      # 명사 추출
      nouns_list = kkma.pos(text)

      filtered_nouns_list = []

      for doc in nouns_list:
          if doc[1] in desired_pos_tags:
            filtered_nouns_list.append(doc[0])


      #크기가 1인 명사 제거
      merged_nouns = [noun for noun in filtered_nouns_list if len(noun) > 1]

      return merged_nouns

    except Exception as e:
        return []



In [10]:
for df in dfs:
    print(f'명사 추출 진행중...')
    df['nouns_list'] = df['Comment'].apply(extract_and_merge_nouns)

명사 추출 진행중...
명사 추출 진행중...
명사 추출 진행중...
명사 추출 진행중...
명사 추출 진행중...
명사 추출 진행중...


In [38]:
dfs[0]

,Unnamed: 0,owner,title,Username,Comment,Likes,Times,Emotion,nouns_list
0,0,YTN,"[속보] 정부 ""5일까지 국가 애도기간"".. 정치권도 위로와 애도 표해 / 이태원...",@user-ss5hd8ou7o,어처구니없는사건 하루종일가슴이먹먹하네요 너무도아까운젊은이들 비통하네요 고인의명복을빕니다,2.0,2022. 10. 29,슬픔,"[사건, 하루, 종일, 가슴, 젊은이, 비통, 고인, 명복]"
1,1,YTN,"[속보] 정부 ""5일까지 국가 애도기간"".. 정치권도 위로와 애도 표해 / 이태원...",@topcule3830,국민들이 바라지도 않은 본인 용와대 출근에는 경찰 인력을 700명이나 동원하면서 수...,37.0,2022. 10. 29,분노,"[국민, 본인, 와대, 출근, 경찰, 인력, 동원, 인파, 예상, 행사, 배치, 이..."
2,2,YTN,"[속보] 정부 ""5일까지 국가 애도기간"".. 정치권도 위로와 애도 표해 / 이태원...",@user-zv6ty8rw2i,국가를 위해서 몸바쳐 돌아가신분들은 아닙니다. 국가애도기간이라니…정말 어이가 없네요...,49.0,2022. 10. 29,슬픔,"[국가, 국가, 애도, 기간, 어이, 국민, 세월, 마음, 마음, 국가, 애도, 기..."
3,3,YTN,"[속보] 정부 ""5일까지 국가 애도기간"".. 정치권도 위로와 애도 표해 / 이태원...",@Akfg-ez7fq,"사전에 통행유도등 통제가\n제대로 이루어 졌다면 \n단,한사람의 생명도 잃지\n않았...",30.0,2022. 10. 29,행복,"[사전, 통행, 유도등, 통제, 사람, 생명]"
4,4,YTN,"[속보] 정부 ""5일까지 국가 애도기간"".. 정치권도 위로와 애도 표해 / 이태원...",@yokool6805,"애도기간이 일주일이나 된다면 순국열사 수준보다 더하니, 책임자 사퇴시켜라. 국민세금...",21.0,2022. 10. 29,분노,"[애도, 기간, 일주일, 순국, 열사, 수준, 책임자, 사퇴, 국민, 세금, 국민,..."
...,...,...,...,...,...,...,...,...,...
1348,221,SBS 뉴스,"11월 5일까지 국가애도기간…""신원미확인자, 17세 미만·외국인 등"" (현장영상) ...",@soundofnature6791,그놈의 세월호도 놀러가다 사고난건데 이젠 누구나 놀다가 사고나서 죽으면 다 보상받고...,NaN,2022. 10. 29,슬픔,"[세월, 호도, 사고, 이젠, 보상, 예우]"
1349,222,SBS 뉴스,"11월 5일까지 국가애도기간…""신원미확인자, 17세 미만·외국인 등"" (현장영상) ...",@user-yb1su3pc6l,청와대를 용산으로 옮기지 안했더라면 많은 보안 인력이 이태원으로 배치해서 이렇게 ...,2.0,2022. 10. 29,슬픔,"[청와대, 보안, 인력, 이태원, 배치, 사망자, 발생]"
1350,223,SBS 뉴스,"11월 5일까지 국가애도기간…""신원미확인자, 17세 미만·외국인 등"" (현장영상) ...",@user-ml8pt1ft3f,삼가고인의 명복을빕니다 그리고 내가족이 사고당했다생각하고 정부가하는 최선의 방칙에 ...,5.0,2022. 10. 29,중립,"[고인, 명복, 가족, 사고, 생각, 정부, 최선, 시비]"
1351,224,SBS 뉴스,"11월 5일까지 국가애도기간…""신원미확인자, 17세 미만·외국인 등"" (현장영상) ...",@user-yb1su3pc6l,너무 젊은 국민들이 많이 사망하여 국가 애도 및 조기계양을 해야 맞다.,NaN,2022. 10. 29,슬픔,"[국민, 사망, 국가, 애도, 조기, 계양]"


In [12]:
emotion_class = ['슬픔', '분노', '행복', '혐오', '중립', '놀람', '공포']

In [40]:
result_dfs = []

for df in dfs:
  # 첫번째 작업: Likes수만큼 nouns_list를 중복시켜서 copy_df에 저장
  copy_df = df.loc[df.index.repeat(df['Likes'].fillna(1))]

  # 두번째 작업: copy_df의 nouns_list에서 가장 많이 등장한 명사 10개 추출(top_nouns)
  nouns_counter = Counter([noun for nouns_list in copy_df['nouns_list'] for noun in nouns_list])
  top_nouns = [noun for noun, _ in nouns_counter.most_common(10)]
  print(nouns_counter.most_common(10))

  # 세번째 작업: top_nouns명사들이 각 Emotion에서 몇번 등장하였는지 아래 형태로 result_df 생성
  result_df = pd.DataFrame(index=top_nouns, columns=copy_df['Emotion'].unique()).fillna(0)
  for _, row in copy_df.iterrows():
      for noun in row['nouns_list']:
          if noun in top_nouns:
              result_df.loc[noun, row['Emotion']] += 1
  result_dfs.append(result_df)

result_dfs[0]

[('애도', 9578), ('국가', 8504), ('기간', 6968), ('사고', 4185), ('사람', 3686), ('나라', 1582), ('고인', 1573), ('명복', 1202), ('참사', 1124), ('예방', 1023)]
[('대통령', 2868), ('사과', 2481), ('국민', 2185), ('천공', 1835), ('책임', 1053), ('사고', 919), ('경찰서', 854), ('하라', 783), ('나라', 696), ('경찰', 616)]
[('경찰', 27681), ('압수', 12286), ('책임', 11405), ('수색', 11212), ('경찰서', 11172), ('현장', 9503), ('국민', 7097), ('수사', 6360), ('안부', 5134), ('대통령', 4699)]
[('유가족', 3043), ('정부', 2461), ('책임', 1979), ('사과', 1688), ('가슴', 1674), ('국민', 1536), ('참사', 1398), ('명복', 1336), ('마음', 1228), ('국가', 1221)]
[('대통령', 1964), ('나라', 1682), ('세월', 1610), ('유가족', 1603), ('사과', 1511), ('책임', 1509), ('사고', 1335), ('국가', 1110), ('국민', 963), ('부모', 830)]
[('특별법', 672), ('사고', 633), ('민주당', 546), ('이태원', 521), ('국민', 514), ('나라', 483), ('눈물', 482), ('참사', 471), ('사람', 424), ('세금', 423)]


,슬픔,분노,행복,혐오,중립,놀람,공포
애도,5060,1345,26,1332,862,443,510
국가,5040,860,85,805,772,463,479
기간,3303,946,16,958,815,454,476
사고,676,411,4,2543,171,297,83
사람,2551,279,30,734,43,3,46
나라,706,281,73,398,13,10,101
고인,1221,49,0,259,43,0,1
명복,1029,63,0,70,39,0,1
참사,489,190,1,27,337,77,3
예방,9,12,0,970,1,0,31


In [42]:
# 결과를 저장할 디렉토리 생성
output_directory = "/content/gdrive/MyDrive/7_topic_3/"
os.makedirs(output_directory, exist_ok=True)

for i, result_df in enumerate(result_dfs):
    prefix = file_prefixes[i]
    output_file_path = f"{output_directory}{prefix}_topic.xlsx"
    result_df.to_excel(output_file_path, index=True)
    print(f"{output_file_path}에 저장 완료")


/content/gdrive/MyDrive/7_topic_3/국가애도기간_topic.xlsx에 저장 완료
/content/gdrive/MyDrive/7_topic_3/대통령사과_topic.xlsx에 저장 완료
/content/gdrive/MyDrive/7_topic_3/경찰책임_topic.xlsx에 저장 완료
/content/gdrive/MyDrive/7_topic_3/정부책임_topic.xlsx에 저장 완료
/content/gdrive/MyDrive/7_topic_3/유가족협의회_topic.xlsx에 저장 완료
/content/gdrive/MyDrive/7_topic_3/특별법_topic.xlsx에 저장 완료
